In [45]:
import os
from os import path
import glob
import pandas as pd
import json
from collections import defaultdict

In [46]:
# perf_dir = "/home/shtoshni/Research/events/models/perf"
perf_dir = "/share/data/speech/shtoshni/research/events/models/perf"
slurm_id_list = ["6421352"]  #"6407936"

files = []

for slurm_id in slurm_id_list:
    files.extend(sorted(glob.glob(path.join(perf_dir, slurm_id + "*")), 
                                  key=lambda x: int(path.splitext(path.basename(x))[0].split('_')[1])))
    
# files = sorted(files, key=lambda x: int(path.splitext(path.basename(x))[0].split('_')[1]))
print(len(files))

30


In [47]:
model_dict_list = []
for file in files:
    model_dict_list.append(json.loads(open(file).read()))

In [48]:
def determine_varying_attributes(model_dict_list, ignore_attribs=['model_dir', 'train', 'test', 'dev', 'pretrained_mention_model',
                                                                  'slurm_id', 'best_model_dir', 'data_dir']):
    attrib_to_vals = defaultdict(set)
    for model_dict in model_dict_list:
        for attrib, val in model_dict.items():
            if attrib in ignore_attribs:
                continue
            else:
                attrib_to_vals[attrib].add(val)
    
    
    varying_attribs = []
    for attrib, vals in attrib_to_vals.items():
        if len(vals) > 1:
            varying_attribs.append(attrib)
            
    return varying_attribs

In [49]:
varying_attribs = determine_varying_attributes(model_dict_list)


In [50]:
perf_df = pd.DataFrame(columns=(varying_attribs))

for model_dict in model_dict_list:
    perf_dict = {}
    for attrib in varying_attribs:
        perf_dict[attrib] = model_dict[attrib]
            
    perf_dict['fscore'] = round(model_dict['dev']['fscore'], 1)
    perf_dict['test_f'] = round(model_dict['test']['fscore'], 1)
                
    perf_df = perf_df.append(perf_dict, ignore_index=True)

In [51]:
idx = perf_df.groupby(['seed', 'sample_invalid', 'max_training_segments'])['fscore'].idxmax()
dev_max = perf_df.iloc[idx]
dev_max

,sample_invalid,max_training_segments,seed,fscore,test_f
0,0.25,3.0,0.0,47.3,47.2
15,0.25,5.0,0.0,49.5,47.5
5,0.50,3.0,0.0,46.9,47.7
20,0.50,5.0,0.0,46.4,47.4
10,0.75,3.0,0.0,47.9,46.8
25,0.75,5.0,0.0,48.6,47.6
1,0.25,3.0,1.0,46.9,48.0
16,0.25,5.0,1.0,47.9,46.7
6,0.50,3.0,1.0,47.7,47.0
21,0.50,5.0,1.0,47.7,46.8


In [53]:
perf_df.groupby(['sample_invalid', 'max_training_segments'])[['fscore', 'test_f']].mean()


fscore  test_f
sample_invalid max_training_segments                
0.25           3.0                     47.24   47.52
               5.0                     48.28   47.40
0.50           3.0                     47.56   47.52
               5.0                     46.66   47.08
0.75           3.0                     46.74   46.84
               5.0                     48.04   47.72